In [2]:
import os
import json
import torch
import argparse
from torch import nn
from expirement import ExpMain
import matplotlib.pyplot as plt
from models.dlinear import DLinear
from dataloader import data_provider
from utills import read_default_args
%load_ext autoreload
%autoreload 2

In [3]:
default_args = read_default_args()

In [4]:
args = {
   'is_training': 1,
   'root_path': '/Users/shiva/Desktop/data_maestro/ltsf',
   'checkpoints': '/Users/shiva/Desktop/data_maestro/checkpoints/',
   'data_path': '03_23.csv',
   'seq_len': 360,
   'pred_len': 90,
   'enc_in': 319,
   'itr': 1,
   'batch_size': 16,
   'learning_rate': 0.05,
   'train_only': False,
   'train_epochs': 10,
   'decomp_kernal': 25
}

for key, value in args.items():
    default_args[key] = value

args = argparse.Namespace(**default_args)

## Training

In [133]:
exp = ExpMain(args)
setting = 'sl{}_pl{}_ch{}_ds_{}'.format(args.seq_len, args.pred_len, args.enc_in, args.data_path.split('.')[0])

Use CPU
Total number of parameters is: 64980


In [134]:
exp.train(setting)

train 10576
val 1486


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 661/661 [01:20<00:00,  8.21it/s]

Epoch: 1 | time: 80.53667402267456


Epoch: 1, Steps: 661 | Train Loss: 1.8737780 Vali Loss: 0.2162994
Validation loss decreased (inf --> 0.216299).  Saving model ...
Updating learning rate to 0.05


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 661/661 [01:15<00:00,  8.75it/s]

...

Epoch: 9 | time: 80.81319999694824


Epoch: 9, Steps: 661 | Train Loss: 0.0497543 Vali Loss: 0.0503004
Validation loss decreased (0.052388 --> 0.050300).  Saving model ...
Updating learning rate to 0.0001953125


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 661/661 [01:17<00:00,  8.48it/s]

Epoch: 10 | time: 77.99766087532043


Epoch: 10, Steps: 661 | Train Loss: 0.0496944 Vali Loss: 0.0504683
EarlyStopping counter: 1 out of 3
Updating learning rate to 9.765625e-05


## Dataloader

In [9]:
data_iter, data_loader = data_provider(args, 'test', return_date=True)

test 2761


In [111]:
a = torch.rand((1,360, 319))
a.shape

torch.Size([1, 360, 319])

In [112]:
class MovingAvg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(MovingAvg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class SeriesDecomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(SeriesDecomp, self).__init__()
        self.moving_avg = MovingAvg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [119]:
sd = SeriesDecomp(51)

In [120]:
sd(a)[0].shape, sd(a)[1].shape

(torch.Size([1, 360, 319]), torch.Size([1, 360, 319]))